In [2]:
!pip install openai-whisper --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 10.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.0 MB/

In [5]:
!pip install gTTS --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 3.7 MB/s eta 0:00:00


In [6]:
!pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00


In [7]:
!pip install soundfile --quiet

In [8]:
!pip install pandas --quiet

In [9]:
import os
import csv
import pandas as pd
from gtts import gTTS
from datetime import datetime

In [23]:
import soundfile as sf
import gradio as gr
import whisper

In [24]:
from gtts import gTTS

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/new electric dataset.csv")

In [38]:
df.head()

,issue_keywords,response
0,"power outage,no electricity,light gone",We are aware of the outage and it will be rest...
1,"bill issue,overbilling,wrong bill",Please provide your reference number to check ...
2,"low voltage,voltage problem,fan not working",Thanks for reporting. A team will inspect the ...
3,"new connection,apply electricity,connection re...",You can apply online at wapda.gov.pk or visit ...
4,"complaint status,reference number,track complaint",Your complaint has been logged. Reference ID: ...


In [11]:
os.makedirs("tts", exist_ok=True)

In [30]:
stt_model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 99.9MiB/s]


In [31]:
def speak(text, path="tts/response.mp3"):
    tts = gTTS(text)
    tts.save(path)
    return path

In [39]:
def detect_intent(user_input):
    user_input = user_input.lower()
    for _, row in df.iterrows():
        keywords = row["issue_keywords"].split(",")
        for keyword in keywords:
            if keyword.strip().lower() in user_input:
                return row["response"]
    return "Sorry, I couldn't understand. Please ask about billing, voltage, or power outage."


In [40]:

def log_interaction(user_input, response):
    with open("wapda_interaction_log.csv", "a", newline='') as file:
        writer = csv.writer(file)
        writer.writerow([datetime.now(), user_input, response])

In [41]:
def voice_chatbot(audio_path):
    try:
        result = stt_model.transcribe(audio_path)
        user_input = result['text']

        bot_reply = detect_intent(user_input)
        voice_path = speak(bot_reply)

        log_interaction(user_input, bot_reply)

        return user_input, bot_reply, voice_path
    except Exception as e:
        return "Error", str(e), None

In [42]:
gr.Interface(
    fn=voice_chatbot,
    inputs=gr.Audio(type="filepath", label="🎙️ Upload your electricity complaint (.mp3, .m4a, .wav)"),
    outputs=[
        gr.Textbox(label="🗣️ You said"),
        gr.Textbox(label="💡 WAPDA Response"),
        gr.Audio(label="🔊 Voice Reply")
    ],
    title="🔌 WAPDA Lahore Grid Station Voice Assistant",
    description="This AI assistant handles common electricity issues like outages, billing, voltage problems, and new connections."
).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://007e253f73f72d3520.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
